In [28]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import numpy as np
%load_ext autoreload
%autoreload 2
from imp import reload
from dataaccessframeworks.read_data import get_movielens, user_filter, training_testing, get_yelp, get_douban, training_testing_XY
from dataaccessframeworks.data_preprocessing import get_one_hot_feature, generate_eval_array
from models.collaborative_filtering import get_user_item_matrix, predict
from models.evaluation import recall_k
from sklearn.preprocessing import normalize
from sklearn.metrics import ndcg_score
import configparser
import wandb
from tqdm import tqdm
from util.mywandb import WandbLog
import util.utility as util

config = configparser.ConfigParser()
config.read(os.path.join(os.path.dirname(os.getcwd()), 'config.ini'))
LIBFM_PATH = '/home/baron/libfm/bin/'
os.environ['LIBFM_PATH'] = LIBFM_PATH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/baron/HW/Recommender_System/util/utility.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sim is 'cos':


In [4]:
reload(data_preprocessing)

NameError: name 'data_preprocessing' is not defined

## 0. Get Data

### MovieLens

In [29]:
import itertools
from random import sample

data = get_movielens()
# str to int
user_movie = np.array([list(map(int, data)) for data in data['user_movie']])
# 濾除使用者評分小於三筆的資料
filter_data = user_filter(user_movie, 0)
print(f"使用者評分大於三次的共有：{filter_data.shape}")
# 取得電影個數及電影個數
len_users, movies = np.unique(filter_data[:,0]), np.unique(filter_data[:,1])
# 取得訓練資料及測試資料
training_data,  testing_data = training_testing(filter_data)
# get uij index
uij_pos, uij_neg = get_uij(training_data, len_users, movies)
print("uij_positive: {}, uij_negative: {}".format(uij_pos.shape, uij_neg.shape))
train_uij = np.vstack((uij_pos, uij_neg))

user_movie:[['196' '242' '3']
 ['186' '302' '3']
 ['22' '377' '1']]
movie_genre:[['1' '3']
 ['1' '4']
 ['1' '5']]
user_age:[['1' '3']
 ['2' '6']
 ['3' '3']]
user_occupation:[['1' '1']
 ['2' '2']
 ['3' '3']]
使用者評分大於三次的共有：(100000, 3)
[0/943] uij_pos: (204, 4), uij_neg: (200, 4)
[300/943] uij_pos: (28345, 4), uij_neg: (28158, 4)
[600/943] uij_pos: (58906, 4), uij_neg: (58516, 4)
[900/943] uij_pos: (85434, 4), uij_neg: (84866, 4)
uij_positive: (88873, 4), uij_negative: (88274, 4)


In [11]:
from scipy.sparse import csr_matrix
from scipy import sparse
from sklearn.preprocessing import LabelEncoder
from dataaccessframeworks.data_preprocessing import get_feature_map

def get_uij_one_hot_feature(data, user_item_col, uij_data, y_col=3, time_col=3, batch_size=10000):
    # 取得user及items feature map 
    users_dict, items_dict, features = get_feature_map(data, user_item_col)

    # 將user item 數值轉為integer
    # user_items = np.array([list(map(int, data))for data in data[user_item_col]])
    # 使用者評分次數小於三筆則剔除
    filter_data = user_filter(uij_data, 0)
    print(filter_data.shape)
    print(filter_data[:5])
    # user label encoder
    le = LabelEncoder()
    filter_data[:, 0] = le.fit_transform(filter_data[:, 0])
    # item label encoder
    ile = LabelEncoder()
    filter_data[:, 1] = ile.fit_transform(filter_data[:, 1])
    filter_data[:, 2] = ile.fit_transform(filter_data[:, 2])
    
    # 做特徵的onehot encoding 
    one_hot_encoder_data, y, concat_data = get_uij_onehot_encoding(filter_data, users_dict, items_dict, features, le, ile, batch_size, y_col)

    return one_hot_encoder_data, y, concat_data

# 取得user及items的one hot encoding map
def get_uij_onehot_encoding(data, users_dict, items_dict, features, le, ile, batch_size, y_col):
    #users_onehot = get_users_onehot(data)
    sparse_, dense = get_uij_feature_onehot(data, users_dict, items_dict, features, le, ile, batch_size)
    
    # 取得y
    y = data[:,y_col].reshape(-1,1)
    
    # return np.concatenate((sparse_, dense), axis=1), y, concat_data
    return sparse.hstack((sparse_, dense), format='csr'), y, data

# 取得feature one hot
def get_uij_feature_onehot(data, users_feature, items_feature, features_map, le, ile, batch_size):
    # 取得user & item個數
    user_number = np.max(data[:,0]) + 1
    item_number = np.max(data[:,1]) + 1
    i_feature = items_feature[1].keys()
    # one hot encoding
    for b in range(0, data.shape[0], batch_size):
        user_one_hot = np.eye(user_number)[data[b:b+batch_size,0]]
        itemi_one_hot = np.eye(item_number)[data[b:b+batch_size,1]]
        itemj_one_hot = np.eye(item_number)[data[b:b+batch_size,2]]
        sparse_ = np.concatenate((user_one_hot, itemi_one_hot, itemj_one_hot), axis=1)
        dense = np.empty((user_one_hot.shape[0], 1), int)

        # create items feature 
        i_feature = items_feature[1].keys()
        for fe in i_feature:
            # sparse
            if fe.split("_")[1] != 'year':
                f_map = features_map[fe]
                feature_lengh = f_map[list(f_map.keys())[0]].shape[1]
                tmp = np.zeros((len(data[b:b+batch_size, 1]), feature_lengh*2))
                for i, item_ij in enumerate(data[b:b+batch_size, 1:3]):
                    item_i, item_j = item_ij
                    item_i = ile.inverse_transform(np.array([item_i])).item()
                    item_j = ile.inverse_transform(np.array([item_j])).item()
                    if item_i not in features_map[fe].keys():
                        # 取第一個鍵值得長度
                        # f_map = features_map[fe]
                        # item_feature_onehot = np.zeros((f_map[list(f_map.keys())[0]].shape[1]))
                        pass
                    else:
                        # item_feature_onehot = features_map[fe][item].toarray()
                        tmp[i, :feature_lengh] = features_map[fe][item_i].toarray()
                    if item_j not in features_map[fe].keys():
                        # 取第一個鍵值得長度
                        # f_map = features_map[fe]
                        # item_feature_onehot = np.zeros((f_map[list(f_map.keys())[0]].shape[1]))
                        pass
                    else:
                        # item_feature_onehot = features_map[fe][item].toarray()
                        tmp[i, feature_lengh:] = features_map[fe][item_j].toarray()
                # sparse_ = np.concatenate((sparse_, tmp), axis=1)
                sparse_ = np.hstack((sparse_, tmp))
            # dense
            else:
                # i = 0
                f_map = features_map[fe]
                feature_lengh = f_map[list(f_map.keys())[0]].shape[1]
                tmp = np.zeros((len(data[b:b+batch_size, 1]), feature_lengh*2))
                for i, item_ij in enumerate(data[b:b+batch_size, 1:3]):
                    item_i, item_j = item_ij
                    item_i = ile.inverse_transform(np.array([item_i])).item()
                    item_j = ile.inverse_transform(np.array([item_j])).item()
                    if item_i not in features_map[fe].keys():
                        # 取第一個鍵值得長度
                        # f_map = features_map[fe]
                        # item_feature_onehot = np.zeros((f_map[list(f_map.keys())[0]].shape[1]))
                        pass
                    else:
                        # item_feature_onehot = features_map[fe][item].toarray()
                        tmp[i, :feature_lengh] = features_map[fe][item_i].toarray()
                    if item_j not in features_map[fe].keys():
                        # 取第一個鍵值得長度
                        # f_map = features_map[fe]
                        # item_feature_onehot = np.zeros((f_map[list(f_map.keys())[0]].shape[1]))
                        pass
                    else:
                        # item_feature_onehot = features_map[fe][item].toarray()
                        tmp[i, feature_lengh:] = features_map[fe][item_j].toarray()
                # dense = np.concatenate((dense, tmp), axis=1)
                dense = np.hstack((dense, tmp))

        # create user feature
        u_feature = users_feature[1].keys()
        for fe in u_feature:
            # sparse
            if fe.split("_")[1] != 'age':
                f_map = features_map[fe]
                tmp = np.zeros((len(data[b:b+batch_size, 1]), f_map[list(f_map.keys())[0]].shape[1]))
                for i, user in enumerate(data[b:b+batch_size, 0]):
                    # i = 0
                    user = le.inverse_transform(np.array([user])).item()
                    if user not in features_map[fe].keys():
                        # 取第一個鍵值得長度
                        # f_map = features_map[fe]
                        # user_feature_onehot = np.zeros((f_map[list(f_map.keys())[0]].shape[1]))
                        pass
                    else:
                        tmp[i] = features_map[fe][user].toarray()
                # sparse_ = np.concatenate((sparse_, tmp), axis=1)
                sparse_ = np.hstack((sparse_, tmp))
                
            # dense
            else:
                f_map = features_map[fe]
                tmp = np.zeros((len(data[b:b+batch_size, 1]), f_map[list(f_map.keys())[0]].shape[1]))
                for i, user in enumerate(data[b:b+batch_size, 0]):
                    # i = 0
                    user = le.inverse_transform(np.array([user])).item()
                    if user not in features_map[fe].keys():
                        # 取第一個鍵值得長度
                        # f_map = features_map[fe]
                        # user_feature_onehot = np.zeros((f_map[list(f_map.keys())[0]].shape[1]))
                        pass
                    else:
                        tmp[i] = features_map[fe][user].toarray()
                # dense = np.concatenate((dense, tmp), axis=1)
                dense = np.hstack((dense, tmp))
        if b==0:
            sparse_matrix = csr_matrix(sparse_)
            dense_matrix = dense
        else:
            sparse_matrix = sparse.vstack((sparse_matrix, csr_matrix(sparse_)))
            dense_matrix = np.vstack((dense_matrix, dense))
        print("[{}/{}] sparse_matrix shape is {}".format(b, data.shape[0], sparse_matrix.shape))
    
    return sparse_matrix, dense_matrix


In [12]:
data = get_movielens()
# str to int
user_movie = np.array([list(map(int, data)) for data in data['user_movie']])
# 濾除使用者評分小於三筆的資料
filter_data = user_filter(user_movie, 0)
print(f"使用者評分大於三次的共有：{filter_data.shape}")
# 取得電影個數及電影個數
len_users, movies = np.unique(filter_data[:,0]), np.unique(filter_data[:,1])
# 取得訓練資料及測試資料
training_data,  testing_data = training_testing(filter_data)
# get uij index
uij_pos, uij_neg = get_uij(training_data, len_users, movies)
print("uij_positive: {}, uij_negative: {}".format(uij_pos.shape, uij_neg.shape))
train_uij = np.vstack((uij_pos, uij_neg))
test_uij_pos, test_uij_neg = get_uij(testing_data, len_users, movies)
print("testing uij_positive: {}, testing uij_negative: {}".format(test_uij_pos.shape, test_uij_neg.shape))
test_uij = np.vstack((test_uij_pos, test_uij_neg))

# normalize rating value
# training_data[:, 2:3] = normalize(training_data[:, 2:3], axis=0)
# testing_data[:, 2:3] = normalize(testing_data[:, 2:3], axis=0)
# train_min = training_data[:, 2:3].min()
# train_max = training_data[:, 2:3].max()
# training_rating = (training_data[:, 2] - train_min)/(train_max-train_min)
# test_min = testing_data[:, 2:3].min()
# test_max = testing_data[:, 2:3].max()
# testing_rating = (testing_data[:, 2:3] - test_min)/(test_max-test_min)
print("users: ", len(len_users))
print("items: ", len(movies))

# generarte one hot encoding
run = 'bpr'
if run=='bpr':
    one_hot_x, y, add_fake_data = get_uij_one_hot_feature(data,  'user_movie', train_uij, batch_size=100000)
else:
    one_hot_x, y, add_fake_data = get_one_hot_feature(data,  'user_movie', batch_size=100000)

X_train, X_test, y_train, y_test = training_testing_XY(one_hot_x, y, random_state=int(config['model']['random_state']))
training_index, test_index, _, _ = training_testing_XY(add_fake_data, y, random_state=int(config['model']['random_state']))
print(X_train.shape, X_test.shape)
print(one_hot_x.shape)

user_movie:[['196' '242' '3']
 ['186' '302' '3']
 ['22' '377' '1']]
movie_genre:[['1' '3']
 ['1' '4']
 ['1' '5']]
user_age:[['1' '3']
 ['2' '6']
 ['3' '3']]
user_occupation:[['1' '1']
 ['2' '2']
 ['3' '3']]
使用者評分大於三次的共有：(100000, 3)
[0/943] uij_pos: (227, 4), uij_neg: (226, 4)
[300/943] uij_pos: (28204, 4), uij_neg: (27978, 4)
[600/943] uij_pos: (58930, 4), uij_neg: (58508, 4)
[900/943] uij_pos: (85758, 4), uij_neg: (85158, 4)
uij_positive: (89138, 4), uij_negative: (88516, 4)
[0/943] uij_pos: (67, 4), uij_neg: (66, 4)
[300/943] uij_pos: (7192, 4), uij_neg: (7018, 4)
[600/943] uij_pos: (15267, 4), uij_neg: (14948, 4)
[900/943] uij_pos: (22326, 4), uij_neg: (21848, 4)
testing uij_positive: (23273, 4), testing uij_negative: (22774, 4)
users:  943
items:  1682
(177654, 4)
[[   1  238 1418    1]
 [   1  176 1504    1]
 [   1  163   54    1]
 [   1  252  795    1]
 [   1  228  179    1]]
[0/177654] sparse_matrix shape is (100000, 4367)
[100000/177654] sparse_matrix shape is (177654, 4367)
(1

### Yelp

In [5]:
from sklearn.preprocessing import LabelEncoder

data = get_yelp()
# str to int
user_business = np.array([list(map(int, data)) for data in data['user_business']])
# 濾除使用者評分小於三筆的資料
filter_data = user_filter(user_business, 0)
# user label encoder
le = LabelEncoder()
filter_data[:, 0] = le.fit_transform(filter_data[:, 0])
filter_data[:, 0] += 1
# item label encoder
ile = LabelEncoder()
filter_data[:, 1] = ile.fit_transform(filter_data[:, 1])
filter_data[:, 1] += 1
# if want to inverse label 
# le.inverse_transform(yelp_training_encoder)
print(f"使用者評分大於三次的共有：{filter_data.shape}")
# 取得business個數及users個數
yelp_users, business = np.unique(filter_data[:,0]), np.unique(filter_data[:,1])
# 取得訓練資料及測試資料
# yelp_training_data,  yelp_testing_data = training_testing(filter_data)
print("users: ", len(yelp_users))
print("items: ", len(business))
# generarte one hot encoding
one_hot_x, y, add_fake_data = get_one_hot_feature(data,  'user_business')
X_train_yelp, X_test_yelp, y_train_yelp, y_test_yelp = training_testing_XY(one_hot_x, y, random_state=int(config['model']['random_state']))
training_index_yelp, test_index_yelp, _, _ = training_testing_XY(add_fake_data, y, random_state=int(config['model']['random_state']))


business_category:[['1', '334', '1'], ['1', '426', '1'], ['2', '211', '1']]
business_city:[['1', '31', '1'], ['2', '35', '1'], ['3', '35', '1']]
user_business:[['1', '8391', '5'], ['1', '8971', '5'], ['2', '186', '5']]
user_compliment:[['2', '1', '1'], ['2', '2', '1'], ['2', '3', '1']]
使用者評分大於三次的共有：(184835, 3)
users:  7326
items:  14127
(184835, 3)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 21453 and the array at index 1 has size 512

### Douban

In [ ]:
# get uij index for bpr
uij_pos, uij_neg = get_uij(np.hstack((X_train, y_train)), users, items)
print("uij_positive: {}, uij_negative: {}".format(uij_pos.shape, uij_neg.shape))
uij = np.vstack(uij_pos, uij_neg)

In [2]:
from sklearn.preprocessing import LabelEncoder

data = get_douban()
# str to int
user_book = np.array([list(map(int, data)) for data in data['user_book']])
# 濾除使用者評分小於三筆的資料
filter_data = user_filter(user_book, 0)
# user label encoder
le = LabelEncoder()
filter_data[:, 0] = le.fit_transform(filter_data[:, 0])
filter_data[:, 0] += 1
# item label encoder
ile = LabelEncoder()
filter_data[:, 1] = ile.fit_transform(filter_data[:, 1])
filter_data[:, 1] += 1
print(f"使用者評分大於三次的共有：{filter_data.shape}")
# 取得business個數及users個數
douban_users, books = np.unique(filter_data[:,0]), np.unique(filter_data[:,1])
# 取得訓練資料及測試資料
# douban_training_data,  douban_testing_data = training_testing(filter_data)
print("users: ", len(douban_users))
print("items: ", len(books))
# generarte one hot encoding
one_hot_x, y, add_fake_data = get_one_hot_feature(data,  'user_book')
X_train_douban, X_test_douban, y_train_douban, y_test_douban = training_testing_XY(one_hot_x, y, random_state=int(config['model']['random_state']))
training_index_douban, test_index_douban, _, _ = training_testing_XY(add_fake_data, y, random_state=int(config['model']['random_state']))


book_author:[['12131', '3871'], ['20995', '10690'], ['9905', '3845']]
book_publisher:[['12131', '108'], ['20995', '1470'], ['9905', '1696']]
book_year:[['9905', '16'], ['21153', '15'], ['12823', '15']]
user_book:[['10855', '938', '4'], ['10027', '3', '3'], ['741', '2426', '5']]
user_group:[['3587', '232'], ['3587', '666'], ['3587', '226']]
user_location:[['3587', '33'], ['3210', '179'], ['7993', '394']]
使用者評分大於三次的共有：(788898, 3)
users:  11266
items:  22347
(788898, 3)
[0/1915498] sparse_matrix shape is (10000, 49626)
[10000/1915498] sparse_matrix shape is (20000, 49626)
[20000/1915498] sparse_matrix shape is (30000, 49626)
[30000/1915498] sparse_matrix shape is (40000, 49626)
[40000/1915498] sparse_matrix shape is (50000, 49626)
[50000/1915498] sparse_matrix shape is (60000, 49626)
[60000/1915498] sparse_matrix shape is (70000, 49626)
[70000/1915498] sparse_matrix shape is (80000, 49626)
[80000/1915498] sparse_matrix shape is (90000, 49626)
[90000/1915498] sparse_matrix shape is (100000

## 1. User-based Collaborative Filtering (U-CF)

In [4]:
import heapq
import copy
from tqdm import tqdm


['/home/baron/HW/Recommender_System/config.ini']

In [14]:
def user_sim_score(users, items, train_data, test_data, k=int(config['CF']['user_K'])):
    # make matrix
    user_matrix = get_user_item_matrix(train_data, users, items)
    test_matrix = get_user_item_matrix(test_data, users, items)
    # 計算bias
    bias_matrix = util.get_bias(user_matrix, users, items)
    # 計算相似度
#     cos, pcc = util.get_sim_array(user_matrix)
#     cosine_dis = cos -  np.identity(len(users))
#     pcc_dis = pcc -  np.identity(len(users))
    
#     sim = {"cos":cosine_dis, "pcc":pcc_dis}
    sim = ["cos", "pcc"]
    evaluation = dict()
    for s in sim:
        delta_list = list()
        predict_array = np.zeros((test_matrix.shape))
        # sim_dis = sim[s]
        sim_array = util.get_sim_array(user_matrix, sim=s)
        sim_dis = sim_array -  np.identity(len(users))
        for i in tqdm(range(len(users)), desc=f"UCF predicting {s} score with {k}"):
            # Suv: 取出前K個最相似的使用者相似度 ex:K=3, output=[0.378, 0.353, 0.336]
            Suv = heapq.nlargest(k ,sim_dis[i])
            # 若i不存在，則跳過
            if np.isnan(sim_dis[i]).all():
                continue
            # top_sim_index: 取出與使用者i最為相似的前K個使用者 ex:K=3, output=[915, 406, 214]
            sim_dis_idx = sim_dis[i].tolist()
            top_sim_index = list(map(sim_dis_idx.index, heapq.nlargest(k,sim_dis[i])))
            # recall
            prediction = list()
            # 計算相似使用者與使用者i的評分誤差
            for item_idx in range(len(items)):
                # 取得使用者i的評分(ground truth)
                rth = test_matrix[i, item_idx]
                # 如果使用者i有進行評分，則才納入計算RMSE
                if rth != 0:
                    # 之後需剔除對電影m未評分的相似使用者，因此先進行複製，才不會影響下一部電影的計算
                    copy_Suv = copy.deepcopy(Suv)
                    # R: 若相似使用者對電影 m 有評分則進行調整
                    R = list()
                    # 判斷相似使用者是否對電影ｍ有評分，若有評分則將原始評分減去該使用者對電影m的bias
                    for c, j in enumerate(top_sim_index):
                        if  test_matrix[j, item_idx] == 0:
                            R.append(0)
                            copy_Suv[c] = 0
                        else:
                            R.append(test_matrix[j, item_idx] - bias_matrix[j, item_idx])
                    # 如果所有相似使用者都沒評分則跳過此次計算
                    if sum(R) != 0:
                        # 預測使用者i對於第m部電影的評分 + 使用者i對電影m的偏差
                        Rui = predict(copy_Suv, R) + bias_matrix[i, item_idx]
                        # 計算square error
                        delta_list.append(util.se(rth, Rui))
                        # 儲存預測結果, 並取四捨五入
                        predict_array[i, item_idx] = Rui
        # 各評估指標
        evaluation[f'{s}_rmse']= util.rmse(delta_list)
        evaluation[f'{s}_recall@10'] = recall_k(test_matrix, predict_array) 
        evaluation[f'{s}_NDCG@10']=ndcg_score(test_matrix, predict_array, k=10)
        
    return evaluation


print("==========\nMovielens:\n==========")
wandb.init(project=config['general']['movielens'],
                        entity=config['general']['entity'],
                        group="UCF")
wandb_log = WandbLog()
movie_reuslt = user_sim_score(len_users, movies, training_data, testing_data)
wandb_log.log_evaluation(movie_reuslt)
print(movie_reuslt)
wandb.finish()

print("==========\nYelp:\n==========")
wandb.init(project=config['general']['yelp'],
                        entity=config['general']['entity'],
                        group="UCF")
wandb_log = WandbLog()
yelp_reuslt = user_sim_score(yelp_users, business, yelp_training_data, yelp_testing_data)
wandb_log.log_evaluation(yelp_reuslt)
print(yelp_reuslt)
wandb.finish()

print("==========\nDouban Book:\n==========")
wandb.init(project=config['general']['douban'],
                        entity=config['general']['entity'],
                        group="UCF")
wandb_log = WandbLog()
douban_reuslt = user_sim_score(douban_users, books, douban_training_data, douban_testing_data)
wandb_log.log_evaluation(douban_reuslt)
print(douban_reuslt)
wandb.finish()

Movielens:


UCF predicting pcc score with 20: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 943/943 [00:03<00:00, 270.53it/s]


{'cos_rmse': 1.185866232216183, 'cos_recall@10': 0.5325477503204228, 'cos_NDCG@10': 0.7979352460670519, 'pcc_rmse': 1.1891346691404043, 'pcc_recall@10': 0.5237494417388817, 'pcc_NDCG@10': 0.7901763769411583}



cos_NDCG@10,0.79794
cos_recall@10,0.53255
cos_rmse,1.18587
pcc_NDCG@10,0.79018
pcc_recall@10,0.52375
pcc_rmse,1.18913


Yelp:


...

cos_NDCG@10,0.09409
cos_recall@10,0.05845
cos_rmse,1.22107
pcc_NDCG@10,0.09333
pcc_recall@10,0.0585
pcc_rmse,1.28634


Douban Book:


data transfer user matrix: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 11266/11266 [00:09<00:00, 1237.99it/s]
/home/baron/HW/Recommender_System/util/utility.py:95: RuntimeWarning: invalid value encountered in true_divide
  return np.reshape(total/exist_number, (-1, 1))
UCF predicting cos score with 20: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 11266/11266 [02:43<00:00, 68.92it/s]
/home/baron/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/baron/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
UCF predicting pcc score with 20: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 11266/11266 [02:44<0

{'cos_rmse': 0.9581459914947442, 'cos_recall@10': 0.17468548559149782, 'cos_NDCG@10': 0.314405097518533, 'pcc_rmse': 0.9643837375362581, 'pcc_recall@10': 0.1729472623095187, 'pcc_NDCG@10': 0.31003647129290945}



cos_NDCG@10,0.31441
cos_recall@10,0.17469
cos_rmse,0.95815
pcc_NDCG@10,0.31004
pcc_recall@10,0.17295
pcc_rmse,0.96438


## 2. Item-based Collaborative Filtering (I-CF)

In [11]:
from scipy import sparse
import pandas as pd
from util.mywandb import WandbLog

def item_sim_score(users, items, train_data, test_data, k=int(config['CF']['user_K'])):
    # make matrix
    user_matrix = get_user_item_matrix(train_data, users, items)
    test_matrix = get_user_item_matrix(test_data, users, items)
    item_matrix = user_matrix.T 
    item_test = test_matrix.T
    #item_test = sparse.csr_matrix(item_test)
    del test_matrix
    
    # 計算bias
    bias_matrix = util.get_bias(user_matrix, users, items)
    item_bias = bias_matrix.T
    del bias_matrix
    del user_matrix
    
    # 計算相似度
    #cos, pcc = util.get_sim_array(item_matrix)
    #cosine_dis = cos -  np.identity(len(items))
    #cosine_dis = sparse.csr_matrix(cosine_dis)
    #pcc_dis = pcc -  np.identity(len(items))
    #pcc_dis = sparse.csr_matrix(pcc_dis)
    #sim = {"cos":cosine_dis, "pcc":pcc_dis}
    sim = ["cos", "pcc"]
    evaluation = dict()
    for s in sim:
        delta_list = list()
        predict_array = np.zeros((item_test.shape))
        # predict array to spase
        predict_array = sparse.csr_matrix(predict_array)
        sim_array = util.get_sim_array(item_matrix, sim=s)
        sim_dis = sim_array -  np.identity(len(items))
        # sim_dis = sim[s]
        for i in tqdm(range(len(items)), desc=f"ICF predicting {s} score with {k}"):
            # Siv: 取出前K個最相似的使用者相似度 ex:K=3, output=[0.378, 0.353, 0.336]
            Siv = heapq.nlargest(k ,sim_dis[i])
            # 若i不存在，則跳過
            if np.isnan(sim_dis[i]).all():
                continue
            sim_dis[i][np.isnan(sim_dis[i])] = 0
            # top_sim_index: 取出與使用者i最為相似的前K個使用者 ex:K=3, output=[915, 406, 214]
            sim_dis_idx = sim_dis[i].tolist()
            top_sim_index = list(map(sim_dis_idx.index, heapq.nlargest(k,sim_dis[i])))
            # recall
            prediction = list()
            # 計算相似電影與電影i的評分誤差
            for user_idx in range(len(users)):
                # 取得項目i的評分(ground truth)
                rth = item_test[i, user_idx]
                # 如果使用者i有進行評分，則才納入計算RMSE
                if rth != 0:
                    # 之後需剔除對電影m未評分的相似使用者，因此先進行複製，才不會影響下一部電影的計算
                    copy_Siv = copy.deepcopy(Siv)
                    # R: 若相似使用者對電影 m 有評分則進行調整
                    R = list()
                    # 判斷相似使用者是否對電影ｍ有評分，若有評分則將原始評分減去該使用者對電影m的bias
                    for c, j in enumerate(top_sim_index):
                        if  item_test[j, user_idx] == 0:
                            R.append(0)
                            copy_Siv[c] = 0
                        else:
                            R.append(item_test[j, user_idx] - item_bias[j, user_idx])
                    # 如果所有相似使用者都沒評分則跳過此次計算
                    if sum(R) != 0:
                        # 預測使用者i對於第m部電影的評分 + 使用者i對電影m的偏差
                        Rui = predict(copy_Siv, R) + item_bias[i, user_idx]
                        # 計算square error
                        delta_list.append(util.se(rth, Rui))
                        # 儲存預測結果, 並取四捨五入
                        if np.isnan(Rui):
                            Rui=0
                        predict_array[i, user_idx] = Rui
        
        
        # 各評估指標
        delta_list = pd.Series(delta_list, dtype=object).fillna(0).tolist()
        evaluation[f'{s}_rmse']= util.rmse(delta_list)
        evaluation[f'{s}_recall@10'] = recall_k(item_test, predict_array) 
        evaluation[f'{s}_NDCG@10']=ndcg_score(item_test, predict_array.toarray(), k=10)
        
    return evaluation

# print("==========\nMovielens:\n==========")
# wandb.init(project=config['general']['movielens'],
#                         entity=config['general']['entity'],
#                         group="ICF")
# wandb_log = WandbLog()
# movie_reuslt = item_sim_score(len_users, movies, training_data, testing_data)
# wandb_log.log_evaluation(movie_reuslt)
# print(movie_reuslt)
# wandb.finish()

print("==========\nYelp:\n==========")
wandb.init(project=config['general']['yelp'],
                        entity=config['general']['entity'],
                        group="ICF")
wandb_log = WandbLog()
yelp_reuslt = item_sim_score(yelp_users, business, yelp_training_data, yelp_testing_data)
wandb_log.log_evaluation(yelp_reuslt)
print(yelp_reuslt)
wandb.finish()

# print("==========\nDouban Book:\n==========")
# wandb.init(project=config['general']['douban'],
#                         entity=config['general']['entity'],
#                         group="ICF")
# wandb_log = WandbLog()
# douban_reuslt = item_sim_score(douban_users, books, douban_training_data, douban_testing_data)
# wandb_log.log_evaluation(douban_reuslt)
# print(douban_reuslt)
# wandb.finish()

Yelp:


ICF predicting cos score with 20:   0%|                                                                                                       | 11/14127 [00:00<02:14, 104.91it/s]/home/baron/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
ICF predicting cos score with 20: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 14127/14127 [02:15<00:00, 103.07it/s]


{'cos_rmse': 1.27345583713731, 'cos_recall@10': 0.00029988668138726505, 'cos_NDCG@10': 0.04081184719369428}


/home/baron/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/baron/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
ICF predicting pcc score with 20:   0%|                                                                                                                 | 0/14127 [00:00<?, ?it/s]/home/baron/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
ICF predicting pcc score with 20: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 14127/14127 [02:30<00:00, 93.69it/s]


{'cos_rmse': 1.27345583713731, 'cos_recall@10': 0.00029988668138726505, 'cos_NDCG@10': 0.04081184719369428, 'pcc_rmse': 1.2473677581038891, 'pcc_recall@10': 0.00029988668138726505, 'pcc_NDCG@10': 0.03952111283588812}
{'cos_rmse': 1.27345583713731, 'cos_recall@10': 0.00029988668138726505, 'cos_NDCG@10': 0.04081184719369428, 'pcc_rmse': 1.2473677581038891, 'pcc_recall@10': 0.00029988668138726505, 'pcc_NDCG@10': 0.03952111283588812}



cos_NDCG@10,0.04081
cos_recall@10,0.0003
cos_rmse,1.27346
pcc_NDCG@10,0.03952
pcc_recall@10,0.0003
pcc_rmse,1.24737


## 3. MF

In [5]:
import wandb
from util.mywandb import WandbLog

# 進行測試資料驗證評估
def test(test_data, p, q, gu=False, bu=False, bi=False):
    rmse_test = list()

    for test in test_data:
        user = test[0] - 1
        movie = test[1] - 1
        # 判斷是否有bias
        if gu and bu.any() and bi.any():
            rmse_test.append(util.se(test[2], (np.dot(p[user], q[movie]) + gu + bu[user] + bi[movie])))
        else:
            rmse_test.append(util.se(test[2], (np.dot(p[user], q[movie]))))
    return util.rmse(rmse_test)

def execute_matrix_factorization(users, items, train_data, test_data):
    # 存放測試資料集的rmse結果
    MF_bias_testing = list()
    # init evaluation
    evaluation = dict()
    user_item = get_user_item_matrix(train_data, users, items)
    test_matrix = get_user_item_matrix(test_data, users, items)

    # init setting global mean
    gu= util.get_u(user_item)
    # init setting user mean as bias
    bu = np.array([util.get_ubias(user_item, i) - gu for i in range(len(users))])
    # init setting items mean as bias
    bi = np.array([util.get_ibias(user_item, m) - gu for m in range(len(items))])

    # init lentent vector
    K = int(config["MF"]["latent_vector_number"])
    # init user lentent matrix
    P = np.random.uniform(low=0, high=3, size=(users.max(), K))
    # init items lentent matrix
    Q = np.random.uniform(low=0, high=3, size=(items.max(), K))

    # parameter
    epochs = int(config["MF"]["epochs"])
    alpha = float(config["MF"]["alpha"])
    l = float(config["MF"]["learning_rate"])

    # 更新次數, init=100
    for epoch in range(epochs):
        # 存放 spuare error 結果
        se_list = list()
        # 針對user有評分過的rating位置進行更新(User Latent Matrix)
        for j in range(len(users)):
            # 找出被使用者j評分過的電影
            # movie_index = [i for i, e in enumerate(user_item[j]) if e != 0]
            movie_index = np.nonzero(user_item[j])[0]
            for m in movie_index:
                # 對u 做偏微分進行ＳＧＤ更新
                tmp_gu = gu - alpha * (((np.dot(P[j], Q[m]) + gu + bu[j] + bi[m]) - user_item[j,m]) + l*(gu))
                # 對bu 做偏微分進行ＳＧＤ更新
                tmp_bu = bu[j] - alpha * (((np.dot(P[j], Q[m]) + gu + bu[j] + bi[m]) - user_item[j,m]) + l*(bu[j]))
                # 對bi 做偏微分進行ＳＧＤ更新
                tmp_bi = bi[m] - alpha * (((np.dot(P[j], Q[m]) + gu + bu[j] + bi[m]) - user_item[j,m]) + l*(bi[m]))
                # 若user item 有值則對Q的相對欄位進行SGD更新, 將更新後user latent matrix先暫存
                tmp = Q[m] - alpha * (((np.dot(P[j], Q[m]) + gu + bu[j] + bi[m]) - user_item[j,m]) * P[j] + l*(Q[m]))
                # 更新 movie latent matrix
                P[j] -= alpha * (((np.dot(P[j], Q[m]) + gu + bu[j] + bi[m]) - user_item[j,m]) * Q[m] + l*(P[j]))
                # 更新 user latent matrix
                Q[m] = tmp
                # 更新bias
                gu = tmp_gu
                bu[j] = tmp_bu
                bi[m] = tmp_bi
                # 計算ＳＥ
                se_list.append(util.se(user_item[j, m], (np.dot(P[j], Q[m]) + gu + bu[j] + bi[m])))
                
        # 進行驗證資料測試
        MF_bias_testing.append(test(test_data, P, Q, gu, bu, bi))
        if epoch % 9 == 0:
            print(f"[{epoch}/{epochs}] gu={gu}, bu={np.mean(bu)}, bi={np.mean(bi)}, testing error={MF_bias_testing[-1]}")

    # 各評估指標
    evaluation['rmse']= MF_bias_testing[-1]
    evaluation['recall@10'] = recall_k(test_matrix, np.dot(P, Q.T))
    evaluation['NDCG@10'] = ndcg_score(test_matrix, np.dot(P, Q.T))
    
    return evaluation

# print("==========\nMovielens:\n==========")
# wandb.init(project=config['general']['movielens'],
#                         entity=config['general']['entity'],
#                         group="MF")
# wandb_log = WandbLog()
# movie_reuslt = execute_matrix_factorization(len_users, movies, training_data, testing_data)
# wandb_log.log_evaluation(movie_reuslt)
# print(movie_reuslt)
# wandb.finish()

print("==========\nYelp:\n==========")
wandb.init(project=config['general']['yelp'],
                        entity=config['general']['entity'],
                        group="MF")
wandb_log = WandbLog()
yelp_reuslt = execute_matrix_factorization(yelp_users, business, yelp_training_data, yelp_testing_data)
wandb_log.log_evaluation(yelp_reuslt)
print(yelp_reuslt)
wandb.finish()

print("==========\nDouban Book:\n==========")
wandb.init(project=config['general']['douban'],
                        entity=config['general']['entity'],
                        group="MF")
wandb_log = WandbLog()
douban_reuslt = execute_matrix_factorization(douban_users, books, douban_training_data, douban_testing_data)
wandb_log.log_evaluation(douban_reuslt)
print(douban_reuslt)
wandb.finish()

Yelp:


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: baron (use `wandb login --relogin` to force relogin)


data transfer user matrix: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7326/7326 [00:01<00:00, 5267.94it/s]
/home/baron/HW/Recommender_System/util/utility.py:97: RuntimeWarning: invalid value encountered in true_divide
  return np.reshape(total/exist_number, (-1, 1))


[0/100] gu=-17.692211953214663, bu=-0.35316409338309057, bi=-0.42891212553119235, testing error=7.955810027961804
[9/100] gu=0.22247643735875575, bu=-0.5249549769093081, bi=-0.5294334345443557, testing error=2.7856859516406733
[18/100] gu=0.5645280880312763, bu=-0.3987602788849377, bi=-0.47826749349173453, testing error=2.4400391999324773
[27/100] gu=0.6296471911712056, bu=-0.29607102872930735, bi=-0.43956784560268547, testing error=2.225455054150513
[36/100] gu=0.6675271534961927, bu=-0.21054304945703303, bi=-0.40919342149140897, testing error=2.069421622171588
[45/100] gu=0.696019974791983, bu=-0.1367402157513489, bi=-0.3840343421814533, testing error=1.9492368710803363
[54/100] gu=0.718561503467009, bu=-0.07161811549490976, bi=-0.3624030291507676, testing error=1.85335723796497
[63/100] gu=0.7366663332675585, bu=-0.013315825634311385, bi=-0.34329892350712354, testing error=1.774947082203483
[72/100] gu=0.7513500769514576, bu=0.03940763190864139, bi=-0.32607865411043097, testing erro

NDCG@10,0.10017
recall@10,0.00011
rmse,1.5662


Douban Book:


data transfer user matrix: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 11266/11266 [00:09<00:00, 1218.46it/s]


[0/100] gu=-9.597066824455768, bu=-0.5293391024078583, bi=-0.3726751178259644, testing error=8.962208379542602
[9/100] gu=1.1956220314470676, bu=-0.06118100359872818, bi=-0.21403802680386308, testing error=1.6258202692057764
[27/100] gu=0.9814044789512671, bu=0.2622510980715556, bi=-0.07807743223896506, testing error=1.1731004316425557
[36/100] gu=0.9607706421336191, bu=0.33401442293151506, bi=-0.04050527084410636, testing error=1.0610577338859197
[45/100] gu=0.9478580213330579, bu=0.38749323080660947, bi=-0.009205749343726458, testing error=0.9795769773050196
[54/100] gu=0.9380314766234154, bu=0.4300271302218055, bi=0.01858431125483541, testing error=0.9182242672602638
[63/100] gu=0.9304860229270434, bu=0.46550845869612617, bi=0.044563510148037735, testing error=0.870964974358609
[72/100] gu=0.925087542232725, bu=0.4963265653534839, bi=0.0699132232794492, testing error=0.8341293347715502
[81/100] gu=0.921647147049031, bu=0.5240367233080105, bi=0.09546750191204248, testing error=0.8053

NDCG@10,0.14308
recall@10,0.0002
rmse,0.76586


## 4. BPR-MF

In [3]:
'''
data input:
[[user, item, rank], .....]
'''
def get_uij(data, users, items, sample_rate=1000):
    for ii, user in enumerate(users):
        items_data = data[data[:, 0]==user]
        item_compare = list()
        item_neg = list()
        neg_count = 0
        items_iter =[i for i in itertools.combinations(items, 2)]
        items_iter = sample(items_iter, sample_rate)
        for i, j in items_iter:
            # if i exist items, but j not exsit items, i>j
            if i in items_data[:, 1] and j not in items_data[:, 1]:
                item_compare.append([user, i, j, 1])
            # if j exist items, but i not exsit items, j>i
            elif i not in items_data[:, 1] and j in items_data[:, 1]:
                item_compare.append([user, j, i, 1])
            # if i exist items, and also j exsit items, compare i and j
            elif i in items_data[:, 1] and j in items_data[:, 1]:
                ri = items_data[(items_data[:, 0]==user) & (items_data[:, 1]==i)][0, 2]
                rj = items_data[(items_data[:, 0]==user) & (items_data[:, 1]==j)][0, 2]
                if ri > rj:
                    item_compare.append([user, i, j, 1])
                elif ri < rj:
                    item_compare.append([user, j, i, 1])
                else:
                    if neg_count < len(item_compare)//2:
                        item_neg.append([user, j, i, 0])
                        item_neg.append([user, i, j, 0])
                        neg_count+=1
            else:
                if neg_count < len(item_compare)//2:
                    item_neg.append([user, j, i, 0])
                    item_neg.append([user, i, j, 0])
                    neg_count+=1
        if ii==0:
            uij = np.array(item_compare)
            uij_neg = np.array(item_neg)
        else:
            if len(item_compare)!= 0:
                uij = np.vstack((uij, np.array(item_compare)))
            if len(item_neg)!= 0:
                uij_neg = np.vstack((uij_neg, np.array(item_neg)))
        
        if ii%300==0:
            print("[{}/{}] uij_pos: {}, uij_neg: {}".format(ii, len(users), uij.shape, uij_neg.shape))
    
    return uij, uij_neg

In [ ]:
import wandb
from util.mywandb import WandbLog
import itertools
from random import sample

# 進行測試資料驗證評估
def test(test_uij, users, items, p, q, gu=False, bu=False, bi=False):
    rmse_test = list()
    
    
    for u, i, j, rank in test_uij:
        u_idx = u - 1
        i_idx = i - 1
        j_idx = j - 1
        rui = np.dot(p[u_idx], q[i_idx])
        ruj = np.dot(p[u_idx], q[j_idx])
        x_uij =  rui - ruj
        # sigmoid
        exp_x = np.exp(-x_uij)
        y_hat = 1/(1 + np.exp(exp_x))
        
        rmse_test.append(util.se(y_hat, rank))
        
    return util.rmse(rmse_test)


            
    
def execute_bpr_matrix_factorization(users, items, train_data, test_data):
    # 存放測試資料集的rmse結果
    MF_bias_testing = list()
    # init evaluation
    evaluation = dict()
    user_item = get_user_item_matrix(train_data, users, items)
    test_matrix = get_user_item_matrix(test_data, users, items)

    # init setting global mean
    gu= util.get_u(user_item)
    # init setting user mean as bias
    bu = np.array([util.get_ubias(user_item, i) - gu for i in range(len(users))])
    # init setting items mean as bias
    bi = np.array([util.get_ibias(user_item, m) - gu for m in range(len(items))])

    # init lentent vector
    K = int(config["MF"]["latent_vector_number"])
    # init user lentent matrix
    # P = np.random.uniform(low=0, high=3, size=(users.max(), K))
    P = np.random.randn(users.max(), K)/10
    # init items lentent matrix
    # Q = np.random.uniform(low=0, high=3, size=(items.max(), K))
    Q = np.random.randn(items.max(), K)/10
    
    # get uij index
    uij_pos, uij_neg = get_uij(train_data, users, items)
    print("uij_positive: {}, uij_negative: {}".format(uij_pos.shape, uij_neg.shape))
    # uij = np.vstack(uij_pos, uij_neg)
    test_uij_pos, test_uij_neg = get_uij(test_data, users, items)
    print("testing uij_positive: {}, testing uij_negative: {}".format(test_uij_pos.shape, test_uij_neg.shape))
    test_uij = np.vstack((test_uij_pos, test_uij_neg))

    # parameter
    epochs = int(config["MF"]["epochs"])
    alpha = float(config["MF"]["alpha"])
    l = float(config["MF"]["learning_rate"])

    # 更新次數, init=100
    for epoch in range(epochs):
        # 針對user有評分過的rating位置進行更新(User Latent Matrix)
        for u, i, j, rank in uij_pos:
            # 計算x_uij
            # rui = np.dot(P[u], Q[i]) + gu + bu[u] + bi[i]
            # ruj = np.dot(P[u], Q[j]) + gu + bu[u] + bi[j]
            i_idx = i-1
            j_idx = j-1
            u_idx = u-1
            rui = np.dot(P[u_idx], Q[i_idx])
            ruj = np.dot(P[u_idx], Q[j_idx])
            x_uij =  rui - ruj
            
            # sigmoid
            exp_x = np.exp(-x_uij)
            partial_BPR = 1/(1 + np.exp(exp_x))
            print("partial_BPR", partial_BPR)
            
            # 更新 user latent matrix
            print(P[u_idx])
            print(P[u_idx].shape)
            print(Q[i_idx]-Q[j_idx])
            New_P= alpha * (partial_BPR*(Q[i_idx]-Q[j_idx]) + l*(P[u_idx]))
            # 若user item 有值則對Q的相對欄位進行SGD更新, 將更新後user latent matrix先暫存
            Q[i_idx] -=  alpha * (partial_BPR*P[u_idx] + l*(Q[i_idx]))
            Q[j_idx] -=  alpha * (partial_BPR*-P[u_idx] + l*(Q[j_idx]))
            # 更新 user latent matrix
            print(New_P)
            P[u_idx] = New_P
            # # 更新bias
            # # 對u 做偏微分進行ＳＧＤ更新
            # gu = gu - alpha * ((rui - user_item[j,m]) + l*(gu))
            # # 對bu 做偏微分進行ＳＧＤ更新
            # bu[j] = bu[j] - alpha * ((rui - user_item[j,m]) + l*(bu[j]))
            # # 對bi 做偏微分進行ＳＧＤ更新
            # bi[m] = bi[m] - alpha * ((rui - user_item[j,m]) + l*(bi[m]))
            break
        break

                
        # 進行驗證資料測試
        MF_bias_testing.append(test(test_uij, users, items, P, Q))
        if epoch % 9 == 0:
            print(f"[{epoch}/{epochs}] testing error={MF_bias_testing[-1]}")
    
    rui = np.dot(P[u_idx], Q[i_idx])
    ruj = np.dot(P[u_idx], Q[j_idx])
    x_uij =  rui - ruj
    print("{} user like item{} more than item{}, score is {}: ".format(u, i, j, x_uij))
    # 各評估指標
    evaluation['rmse']= MF_bias_testing[-1]
    evaluation['recall@10'] = recall_k(test_matrix, np.dot(P, Q.T))
    evaluation['NDCG@10'] = ndcg_score(test_matrix, np.dot(P, Q.T))
    
    return evaluation, P, Q

# print("==========\nMovielens:\n==========")
# wandb.init(project=config['general']['movielens'],
#                         entity=config['general']['entity'],
#                         group="BPR-MF")
# wandb_log = WandbLog()
movie_reuslt, P, Q = execute_bpr_matrix_factorization(len_users, movies, training_data, testing_data)
print(movie_reuslt)
# wandb_log.log_evaluation(movie_reuslt)
# wandb.finish()

# print("==========\nYelp:\n==========")
# wandb.init(project=config['general']['yelp'],
#                         entity=config['general']['entity'],
#                         group="MF")
# wandb_log = WandbLog()
# yelp_reuslt = execute_matrix_factorization(yelp_users, business, yelp_training_data, yelp_testing_data)
# wandb_log.log_evaluation(yelp_reuslt)
# print(yelp_reuslt)
# wandb.finish()

# print("==========\nDouban Book:\n==========")
# wandb.init(project=config['general']['douban'],
#                         entity=config['general']['entity'],
#                         group="MF")
# wandb_log = WandbLog()
# douban_reuslt = execute_matrix_factorization(douban_users, books, douban_training_data, douban_testing_data)
# wandb_log.log_evaluation(douban_reuslt)
# print(douban_reuslt)
# wandb.finish()

Movielens:


data transfer user matrix: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 943/943 [00:00<00:00, 9318.19it/s]


[0/943] uij_pos: (217, 4), uij_neg: (216, 4)
[300/943] uij_pos: (28474, 4), uij_neg: (28262, 4)
[600/943] uij_pos: (58943, 4), uij_neg: (58548, 4)
[900/943] uij_pos: (85906, 4), uij_neg: (85332, 4)
uij_positive: (89328, 4), uij_negative: (88726, 4)
[0/943] uij_pos: (66, 4), uij_neg: (66, 4)
[300/943] uij_pos: (7351, 4), uij_neg: (7186, 4)
[600/943] uij_pos: (15536, 4), uij_neg: (15218, 4)


## 5. FM

In [3]:
import pywFM
import random
from IPython.display import clear_output

def execute_factorization_machine(X, y, X_test, y_test, training_index, test_index, users, items):
    rating_testing_array = generate_eval_array(y_test, test_index, users, items)
    
    # kfold = 5
    kfold = list()
    recall = list()
    ndcg = list()
    result = dict()
    sum_predict_values = 0 
    for i in range(5):
        print(f"Start {i} FM Cross-Validation")
        random_state = random.randint(0, 50)
        X_train, X_val, y_train, y_val = training_testing_XY(X, y, test_size=float(config["model"]["val_rate"]), random_state=random_state)
        _, val_index, _, _ = training_testing_XY(training_index, y, test_size=float(config["model"]["val_rate"]), random_state=random_state)

        # reshape y
        y_train = y_train.reshape(1, -1)[0]
        y_test = y_test.reshape(1, -1)[0]
        y_val = y_val.reshape(1, -1)[0]

        # define model
        fm = pywFM.FM(task='regression')

        model = fm.run(X_train, y_train, X_val, y_val)
        predict_values = model.predictions
        predict = generate_eval_array(predict_values, val_index, users, items)
        kfold.append(util.rmse(predict_values - y_val))
        recall.append(recall_k(rating_testing_array, predict))
        ndcg.append(ndcg_score(rating_testing_array, predict))
        #sum_predict_values += predict_values
        clear_output()

    result['rmse'] = sum(kfold)/len(kfold) 
    result['recall@10'] = sum(recall)/len(recall)
    result['NDCG@10'] = sum(ndcg)/len(ndcg)

    return result

# print("==========\nMovielens:\n==========")
# wandb.init(project=config['general']['movielens'],
#                         entity=config['general']['entity'],
#                         group="FM")
# wandb_log = WandbLog()
# movie_reuslt = execute_factorization_machine(X_train, y_train, X_test, y_test, training_index, test_index, len_users, movies)

# wandb_log.log_evaluation(movie_reuslt)
# print(movie_reuslt)
# wandb.finish()

# print("==========\nYelp:\n==========")
# wandb.init(project=config['general']['yelp'],
#                         entity=config['general']['entity'],
#                         group="FM")
# wandb_log = WandbLog()
# yelp_reuslt = execute_factorization_machine(X_train_yelp, y_train_yelp, X_test_yelp, y_test_yelp, training_index_yelp, test_index_yelp, yelp_users, business)

# wandb_log.log_evaluation(yelp_reuslt)
# print(yelp_reuslt)
# wandb.finish()

print("==========\nDouban Book:\n==========")
wandb.init(project=config['general']['douban'],
                        entity=config['general']['entity'],
                        group="FM")
wandb_log = WandbLog()
douban_reuslt = execute_factorization_machine(X_train_douban, y_train_douban, X_test_douban, y_test_douban, training_index_douban, test_index_douban, douban_users, books)
wandb_log.log_evaluation(douban_reuslt)
print(douban_reuslt)
wandb.finish()

Start 1 FM Cross-Validation


KeyboardInterrupt: 

## 6. BPR-FM

In [26]:
import pywFM
import random
from IPython.display import clear_output

def execute_bpr_factorization_machine(X, y, X_test, y_test, training_index, test_index, users, items):
    rating_testing_array = generate_eval_array(y_test, test_index, users, items)
    
    # kfold = 5
    kfold = list()
    recall = list()
    ndcg = list()
    result = dict()
    sum_predict_values = 0 
    for i in range(5):
        print(f"Start {i} FM Cross-Validation")
        random_state = random.randint(0, 50)
        X_train, X_val, y_train, y_val = training_testing_XY(X, y, test_size=float(config["model"]["val_rate"]), random_state=random_state)
        _, val_index, _, _ = training_testing_XY(training_index, y, test_size=float(config["model"]["val_rate"]), random_state=random_state)
        
        # reshape y
        y_train = y_train.reshape(1, -1)[0]
        y_test = y_test.reshape(1, -1)[0]
        y_val = y_val.reshape(1, -1)[0]

        # define model
        fm = pywFM.FM(task='classification')

        model = fm.run(X_train, y_train, X_val, y_val)
        predict_values = model.predictions
        predict = generate_eval_array(predict_values, val_index, users, items)
        print(predict_values - y_val)
        kfold.append(util.rmse(list(map(abs, predict_values - y_val))))
        recall.append(recall_k(rating_testing_array, predict))
        ndcg.append(ndcg_score(rating_testing_array, predict))
        #sum_predict_values += predict_values
        clear_output()

    result['rmse'] = sum(kfold)/len(kfold) 
    result['recall@10'] = sum(recall)/len(recall)
    result['NDCG@10'] = sum(ndcg)/len(ndcg)

    return result

print("==========\nMovielens:\n==========")
wandb.init(project=config['general']['movielens'],
                        entity=config['general']['entity'],
                        group="BPR-FM")
wandb_log = WandbLog()
movie_reuslt = execute_bpr_factorization_machine(X_train, y_train, X_test, y_test, training_index, test_index, len_users, movies)

wandb_log.log_evaluation(movie_reuslt)
print(movie_reuslt)
wandb.finish()

# print("==========\nYelp:\n==========")
# wandb.init(project=config['general']['yelp'],
#                         entity=config['general']['entity'],
#                         group="BPR-FM")
# wandb_log = WandbLog()
# yelp_reuslt = execute_factorization_machine(X_train_yelp, y_train_yelp, X_test_yelp, y_test_yelp, training_index_yelp, test_index_yelp, yelp_users, business)

# wandb_log.log_evaluation(yelp_reuslt)
# print(yelp_reuslt)
# wandb.finish()

# print("==========\nDouban Book:\n==========")
# wandb.init(project=config['general']['douban'],
#                         entity=config['general']['entity'],
#                         group="BPR-FM")
# wandb_log = WandbLog()
# douban_reuslt = execute_factorization_machine(X_train_douban, y_train_douban, X_test_douban, y_test_douban, training_index_douban, test_index_douban, douban_users, books)
# wandb_log.log_evaluation(douban_reuslt)
# print(douban_reuslt)
# wandb.finish()

{'rmse': 0.5321011130501097, 'recall@10': 0.03858693938782245, 'NDCG@10': 0.3017260812416944}



NDCG@10,0.30173
recall@10,0.03859
rmse,0.5321
